In this notebook one can:


In [54]:
import sys
sys.path.append('/dls_sw/e02/software/epsic_tools')
import epsic_tools.api as ep
import pprint
import re
import subprocess
import os
from __future__ import print_function,unicode_literals
import subprocess
import glob
import json
import h5py
import numpy as np

In [55]:
base_path = '/dls/e02/data/2023/cm33902-5/processing/Merlin/tile_graphene_ptycho_multi'
template_json = '/dls/e02/data/2023/mg35540-1/processing/Merlin/graphene_mosaic_20Mx_x30_y80_pieze/20231004_162104/pty_out/inital_recons/Mosaic_graphene_20Mx_162104.json'

In [56]:
meta_files = glob.glob(base_path+'/*/*.hdf')

In [58]:
for i, file in enumerate(meta_files):
    print(i, file)

0 /dls/e02/data/2023/cm33902-5/processing/Merlin/tile_graphene_ptycho_multi/20231031_120836/20231031_120836.hdf
1 /dls/e02/data/2023/cm33902-5/processing/Merlin/tile_graphene_ptycho_multi/20231031_121557/20231031_121557.hdf
2 /dls/e02/data/2023/cm33902-5/processing/Merlin/tile_graphene_ptycho_multi/20231031_122419/20231031_122419.hdf
3 /dls/e02/data/2023/cm33902-5/processing/Merlin/tile_graphene_ptycho_multi/20231031_121659/20231031_121659.hdf
4 /dls/e02/data/2023/cm33902-5/processing/Merlin/tile_graphene_ptycho_multi/20231031_122934/20231031_122934.hdf
5 /dls/e02/data/2023/cm33902-5/processing/Merlin/tile_graphene_ptycho_multi/20231031_121351/20231031_121351.hdf
6 /dls/e02/data/2023/cm33902-5/processing/Merlin/tile_graphene_ptycho_multi/20231031_121042/20231031_121042.hdf
7 /dls/e02/data/2023/cm33902-5/processing/Merlin/tile_graphene_ptycho_multi/20231031_123037/20231031_123037.hdf
8 /dls/e02/data/2023/cm33902-5/processing/Merlin/tile_graphene_ptycho_multi/20231031_121905/20231031_121

In [59]:
# Load json file and modify
# config_file = '/dls/e02/data/2023/mg35540-1/processing/Merlin/graphene_mosaic_20Mx_x30_y80_pieze/20231004_105110/pty_out/inital_recons/Mosaic_graphene_20Mx_105110.json'
with open(template_json,'r') as template_file:
    pty_expt = json.load(template_file)

In [60]:
pty_expt

{'base_dir': '/dls/e02/data/2023/mg35540-1/processing/Merlin/graphene_mosaic_20Mx_x30_y80_pieze/20231004_162104/pty_out/inital_recons',
 'experiment': {'data': {'dask': 1,
   'data_key': 'data/frames',
   'data_path': '/dls/e02/data/2023/mg35540-1/processing/Merlin/graphene_mosaic_20Mx_x30_y80_pieze/20231004_162104/20231004_162104.hdf',
   'dead_pixel_flag': 1,
   'dead_pixel_path': '/dls/science/groups/e02/Frederick/DetecotrMasks/Mask11082023a.h5',
   'flat_field_flag': 0,
   'flat_field_path': 'None',
   'load_flag': 1,
   'meta_type': 'hdf'},
  'detector': {'position': [0, 0, 0.124]},
  'experiment_ID': '80keV',
  'optics': {'lens': {'alpha': 0.0496,
    'defocus': [-8.500000000000005e-09, -8.500000000000005e-09],
    'use': 1}},
  'sample': {'position': [0, 0, 0]}},
 'process': {'PIE': {'decay': [1, 1, 1],
   'detector': {'border': [0, 0], 'border_float': 1, 'upsample': [1, 1]},
   'iterations': 200,
   'object': {'alpha': 0.1, 'end': -1, 'start': 1},
   'probe': {'alpha': 0.2, 'en

In [61]:
def gen_config(template_json, dest_path, config_name, meta_file_path):
    config_file = dest_path + '/' + config_name + '.json'

    with open(template_json,'r') as template_file:
        pty_expt = json.load(template_file)
    data_path = meta_file_path

    pty_expt['base_dir'] = dest_path
    pty_expt['process']['save_dir'] = dest_path
    pty_expt['experiment']['data']['data_path'] = data_path

    # pty_expt['process']['common']['scan']['rotation'] = rotation_angle


    # pty_expt['process']['common']['scan']['N'] = scan_shape
    # pty_expt['experiment']['detector']['position'] = [0, 0, camera_length]
    # pty_expt['experiment']['optics']['lens']['alpha'] = conv_angle


    with h5py.File(meta_file_path,'r') as microscope_meta:
        meta_values = microscope_meta['metadata']
        pty_expt['process']['common']['scan']['N'] = [int(meta_values['4D_shape'][:2][0]), int(meta_values['4D_shape'][:2][1])]
        pty_expt['process']['common']['source']['energy'] = [float(np.array(meta_values['ht_value(V)']))]
        pty_expt['process']['PIE']['source']['sx'] = 3
        # pty_expt['process']['common']['scan']['dR'] = [float(np.array(meta_values['step_size(m)'])), float(np.array(meta_values['step_size(m)']))]
        # pty_expt['experiment']['optics']['lens']['alpha'] = 2 * float(np.array(meta_values['convergence_semi-angle(rad)']))
        # pty_expt['experiment']['optics']['lens']['defocus'] = [float(np.array(meta_values['defocus(nm)'])*1e-9), float(np.array(meta_values['defocus(nm)'])*1e-9)]
        pty_expt['process']['save_prefix'] = config_name

 

    
    with open(config_file,'w') as f:
        json.dump(pty_expt, f, indent=4)

In [62]:
file_index = 10

In [64]:
base_dir = os.path.dirname(meta_files[file_index])
ptyrex_dir = os.path.join(base_dir, 'ptyrex_recon')
if not os.path.exists(ptyrex_dir):
    os.makedirs(ptyrex_dir)
time_stamp = os.path.basename(meta_files[file_index]).split('.')[0]
gen_config(template_json, ptyrex_dir, time_stamp, meta_files[file_index])

In [65]:
time_stamp

'20231031_120116'

In [66]:
# Create a bash script to submit to SLURM 
bash_script_path = os.path.join(ptyrex_dir, 'cluster_submit.sh')
new_json_path = os.path.join(ptyrex_dir, f'{time_stamp}.json')
with open (bash_script_path, 'w') as f:
    f.write('''#!/usr/bin/env bash
#SBATCH --partition=cs05r
#SBATCH --job-name=ptyrex_recon
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=4
#SBATCH --cpus-per-task=1
#SBATCH --time=08:00:00
#SBATCH --gpus-per-node=4
#SBATCH --mem=60G
'''
f"#SBATCH --error={ptyrex_dir}{os.sep}error_%j.out\n"
f"#SBATCH --output={ptyrex_dir}{os.sep}output_%j.out\n"
f"module load ptyrex/latest\n"
f"mpirun -np 4 ptyrex_recon -c {new_json_path}\n"
           )

In [67]:

sshProcess = subprocess.Popen(['ssh',
                               '-tt',
                               'wilson'],
                               stdin=subprocess.PIPE, 
                               stdout = subprocess.PIPE,
                               universal_newlines=True,
                               bufsize=0)
sshProcess.stdin.write("ls .\n")
sshProcess.stdin.write("echo END\n")
sshProcess.stdin.write(f"sbatch {bash_script_path}\n")
sshProcess.stdin.write("uptime\n")
sshProcess.stdin.write("logout\n")
sshProcess.stdin.close()


for line in sshProcess.stdout:
    if line == "END\n":
        break
    print(line,end="")

#to catch the lines up to logout
for line in  sshProcess.stdout: 
    print(line,end="")


 _______   __        ______         __    __  _______    ______
|       \ |  \      /      \       |  \  |  \|       \  /      \
| $$$$$$$\| $$     |  $$$$$$\      | $$  | $$| $$$$$$$\|  $$$$$$\
| $$  | $$| $$     | $$___\$$      | $$__| $$| $$__/ $$| $$   \$$
| $$  | $$| $$      \$$    \       | $$    $$| $$    $$| $$
| $$  | $$| $$      _\$$$$$$\      | $$$$$$$$| $$$$$$$ | $$   __
| $$__/ $$| $$_____|  \__| $$      | $$  | $$| $$      | $$__/  \
| $$    $$| $$     \\$$    $$      | $$  | $$| $$       \$$    $$
 \$$$$$$$  \$$$$$$$$ \$$$$$$        \$$   \$$ \$$        \$$$$$$

Welcome To Wilson - The DLS Slurm Cluster
For Help or Support - Visit: https://schelpdesk.diamond.ac.uk

Please Refrain From Running Any Tasks Directly On This Node - It Is A Submission Node Only

For jobs needing to access the GPFS02 filesystem please use the cs04r partition, for the GPFS03 filesystem please use the cs05r partition status

To get an interactive session on a GPFS02 node enter "iact02", for a GPF

Connection to wilson closed.


In [68]:
pwd

'/dls/science/groups/e02/Mohsen/code/jupyterhub_active/SLURM_notebook_submit'